<a href="https://colab.research.google.com/github/hiroalchem/data_science_lecture_2023/blob/main/Day4_20230308_Cell_Classification_Using_Deep_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 今日の取り組み   
今日はTensorFlowを用いた基本的な画像分類を行ってみます


In [ ]:
# まずデータをダウンロードして解凍します
import gdown
gdown.download('https://drive.google.com/uc?id=1ftQvsAllb457U_bWKVIOLHhr_fr31sRu', 'CellCycle_for_classification.zip', quiet=False)
!unzip CellCycle_for_classification.zip

In [ ]:
# lectureのディレクトリに移動します
%cd /content/CellCycle_for_classification

In [ ]:
!pwd

# **0. 準備**

## 0-1: 必要なライブラリを読み込みます

In [ ]:
!pip install tensorflow==2.8.3

In [ ]:
import os
from pathlib import Path

from skimage import io
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.utils import plot_model
from sklearn.metrics import confusion_matrix, classification_report

## 0-2:  フォルダ構造を確認します
train, val, testのフォルダがあり、その中にG1, G2, Prophaseのフォルダがあることを確認します。    
Day4    
&emsp;  |- train   
&emsp;  |&emsp;    |-G1   
&emsp;  |&emsp;    |-G2  
&emsp;  |&emsp;    |-Prophase  
&emsp;  |   
&emsp;  |- val   
&emsp;  |&emsp;    |-G1   
&emsp;  |&emsp;    |-G2  
&emsp;  |&emsp;    |-Prophase  
&emsp;  |   
&emsp;  |- test   
&emsp;   &emsp;    |-G1   
&emsp;   &emsp;    |-G2  
&emsp;   &emsp;    |-Prophase  

In [ ]:
# train, validation, testフォルダのパスを変数に格納しておきます
root_dir = ('./')
train_dir = os.path.join(root_dir, 'train')
val_dir = os.path.join(root_dir, 'val')
test_dir = os.path.join(root_dir, 'test')


# **1. データとモデルの準備**

## 1-1: データをTensorFlowが使える形に読み込みます
Tensorflowは大きく2つの方法を用意してくれています。   
・ フォルダから直接読み込む   
・ 先に画像を別のライブラリで読み込んでおいてからTensorFlowの形式に変換する   
今回は前者でやってみます


In [ ]:
# まずバッチサイズと画像サイズを指定します。
# バッチサイズとは一度にモデルに渡す画像の枚数で、これが大きいほど学習は安定します (epochごとに渡す枚数ではなく、epoch内で小分けにする画像の枚数です)
# バッチサイズはメモリが許す限り大きい方が望ましいです。今回は32にしてみます。

BATCH_SIZE = 32

# 画像サイズはリサイズした後のサイズになります。今回は元画像が 66x66になっているのでそのまま66x66で使用します。

IMG_SIZE = (66, 66)



train_dataset = image_dataset_from_directory(train_dir,
                                             shuffle=True,
                                             batch_size=BATCH_SIZE,
                                             image_size=IMG_SIZE,
                                             label_mode='categorical')
val_dataset = image_dataset_from_directory(val_dir,
                                             shuffle=True,
                                             batch_size=BATCH_SIZE,
                                             image_size=IMG_SIZE,
                                             label_mode='categorical')
test_dataset = image_dataset_from_directory(test_dir,
                                             shuffle=False,
                                             batch_size=1,
                                             image_size=IMG_SIZE,
                                             label_mode='categorical')

## 1-2: 画像のチェック

In [ ]:
# class名を取得します
class_names = train_dataset.class_names
print(class_names)

In [ ]:
# 学習用データセットから9枚表示してみます。
plt.figure(figsize=(10, 10)) # 描画するfigの準備
for images, labels in train_dataset.take(1): # データセットからバッチを取り出します
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1) # 3x3マスのどこに描画するか指定
    plt.imshow(images[i].numpy().astype("uint8")) # 画像をnumpy形式に変換し、かつuint8に変換して表示
    plt.title(class_names[np.argmax(labels[i])]) # その画像のラベルを取得する
    plt.axis("off")

In [ ]:
# 学習時間を短縮するために、学習時のデータの読み込みを並列化させる設定を行なっておきます。
AUTOTUNE = tf.data.AUTOTUNE

train_dataset = train_dataset.prefetch(buffer_size=AUTOTUNE)
val_dataset = val_dataset.prefetch(buffer_size=AUTOTUNE)
test_dataset = test_dataset.prefetch(buffer_size=AUTOTUNE)

## 1-3: 画像のaugmentation (水増し) の準備
深層学習ではaugmentationという手法を使ってデータの水増しを行います。   
データになんらかの画像処理を加えて見た目を変えた画像を学習に追加することで、モデルの汎化性能を向上させることができます。

In [ ]:
# ここではhorizontal flipとrotationを追加してみます
data_augmentation = tf.keras.Sequential([
  tf.keras.layers.experimental.preprocessing.RandomFlip('horizontal'),
  tf.keras.layers.experimental.preprocessing.RandomRotation(0.2),
])

In [ ]:
# augmentationの結果を確認してみます
for image, _ in train_dataset.take(1): 
  plt.figure(figsize=(10, 10))
  first_image = image[0] # 最初の画像を取り出します
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    augmented_image = data_augmentation(tf.expand_dims(first_image, 0)) # 最初の画像をaugmentationにかけて結果を得ます
    plt.imshow(augmented_image[0] / 255) # 結果を表示
    plt.axis('off')

## 1-4: モデルの準備
今回はResNet50というモデルを使用します。   
ResNetは古くからあるモデルですが現在も改良されながら使われています。   
検索するとたくさんの情報が得られるので、興味のある方は調べてみてください。   

In [ ]:
# まずResNet50を呼び出します。
# TensorFlow (Keras) では、いくつかのモデルが簡単に使えるようになっています。
# 入力する画像の形状の準備します。
IMG_SHAPE = IMG_SIZE + (3,)
print(IMG_SHAPE)

# モデルをbase_modelとして準備します。
# weights='imagenet' とすることで、imagenetで事前学習された重みを取得できます。
base_model = tf.keras.applications.ResNet50(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')

In [ ]:
# base_modelを通して出てくる特徴量の形状を確認してみましょう。
image_batch, label_batch = next(iter(train_dataset))
feature_batch = base_model(image_batch)
print(feature_batch.shape)

In [ ]:
# modelのsummaryを表示することもできます。
base_model.summary()

In [ ]:
# モデル構造をみやすいプロットにすることもできます。
plot_model(base_model,show_shapes=True, to_file='base_model.png')

In [ ]:
# 次に前処理を準備します。
# 画像の前処理としてimagenetで学習された際のresnet50の前処理を取得します。
preprocess_input = tf.keras.applications.resnet50.preprocess_input

In [ ]:
# 最終的な出力までモデルを構築します。
inputs = tf.keras.Input(shape=(66, 66, 3)) # inputの指定
x = data_augmentation(inputs) # まずaugmentationをします。
x = preprocess_input(x) # 前処理をします。
x = base_model(x, training=False) # base_modelを通します。
x = tf.keras.layers.GlobalAveragePooling2D()(x) # gobal average poolingという手法で3x3x2048次元を 2048次元に落とします。
x = tf.keras.layers.Dropout(0.2)(x) # dropoutを追加します。
outputs = tf.keras.layers.Dense(3, activation='softmax')(x) # 最後にクラス数と同じ3次元まで落とし、softmax関数を通します。
model = tf.keras.Model(inputs, outputs) # モデルのinputとoutputを指定して終了です。

In [ ]:
# 最終的なモデルを確認しましょう。
plot_model(model,show_shapes=True, to_file='model.png')

In [ ]:
# まずbase_modelは学習をさせず、追加した部分だけを学習させてみます。
base_model.trainable = False

In [ ]:
# 学習率を指定
base_learning_rate = 0.0001

# モデルをコンパイルする必要があります。
# 最適化アルゴリズムはAdamにしてみます。こ
# 損失関数は Categorical Crossentropyです。これは3クラス以上の分類に使用する基本的な損失関数です。
# metricsは評価のために使用する評価関数です。今回はaccuracyを指定してみましょう。
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=base_learning_rate),
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
# 最後にモデルをチェックします。
model.summary()

In [ ]:
# 学習に使用するパラメータ (層の数) をチェック
len(model.trainable_variables)

# **2. 学習と評価**

## 2-1 準備したモデルを学習させてみます

In [ ]:
# まず10epoch学習させてみましょう。
initial_epochs = 10

In [ ]:
# 学習前のlossとaccuracyを調べます。
# 学習前なのでvalidtaion datasetに対するaccuracyはほぼランダムな場合の数値になると思います。

loss0, accuracy0 = model.evaluate(val_dataset)

In [ ]:
# では学習させましょう。
# 学習結果はhistoryに保存されます。
history = model.fit(train_dataset, # train datasetの指定
                    epochs=initial_epochs, # 学習するepoch数の指定
                    validation_data=val_dataset) # validation datasetの指定

In [ ]:
# 学習結果をプロットしてみます。
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.ylim([0,2.0])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

## 2-2: テストデータで評価します

In [ ]:
# test_datasetから画像とlabelを取り出して、モデルで推論させます。
predictions = np.array([])
labels =  np.array([])
for x, y in test_dataset:
  predictions = np.concatenate([predictions, np.argmax(model.predict(x), axis = -1)])
  labels = np.concatenate([labels, np.argmax(y.numpy(), axis=-1)])

In [ ]:
# 混同行列を表示します
cm = confusion_matrix(labels, predictions)
pd.DataFrame(cm,columns=["pred_" + str(n) for n in class_names], index=["GT_" + str(n) for n in class_names])

In [ ]:
# 行ごとに割合にして表示してみましょう。
pd.DataFrame(cm,columns=["pred_" + str(n) for n in class_names], index=["GT_" + str(n) for n in class_names]).apply(lambda x:x/sum(x),axis=1)

In [ ]:
# 次に各々のクラスについて、precision, recall, f1-scoreを表示してみましょう。
print(classification_report(labels, predictions, target_names=class_names))

## 2-3: 畳み込み層も学習させてみる
ここまではResNetの中は学習させませんでした。   
今度はResNetの一部の層も学習させてみましょう。

In [ ]:
# base_modelを学習可能にします。
base_model.trainable = True
print("base_modelの層の数は ", len(base_model.layers))

In [ ]:
# 最後の10層だけ学習せてみます。
fine_tune_at = -10

# 最後の10層よりも前の層は学習させません。
for layer in base_model.layers[:fine_tune_at]:
  layer.trainable = False
for layer in base_model.layers[fine_tune_at:]:
  print(layer, layer.trainable)

In [ ]:
# コンパイルします。
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=base_learning_rate),
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
len(model.trainable_variables)

In [ ]:
# 10epoch追加で学習させます。
fine_tune_epochs = 10

# 前回のepoch数と足します。
# モデルの総epoch数は20になります。
total_epochs =  initial_epochs + fine_tune_epochs
print(total_epochs)

In [ ]:
# 学習
history_fine = model.fit(train_dataset,
                         epochs=total_epochs, # epoch数はtotal epochsです。
                         initial_epoch=history.epoch[-1], # 前回の学習結果から再開するため、ここでhistoryに保存されているepoch数を呼び出します。
                         validation_data=val_dataset)

In [ ]:
acc_fine = acc + history_fine.history['accuracy']
val_acc_fine = val_acc + history_fine.history['val_accuracy']

loss_fine = loss + history_fine.history['loss']
val_loss_fine = val_loss + history_fine.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc_fine, label='Training Accuracy')
plt.plot(val_acc_fine, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss_fine, label='Training Loss')
plt.plot(val_loss_fine, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.ylim([0,2.0])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

In [ ]:
# test_datasetから画像とlabelを取り出して、モデルで推論させます。
predictions = np.array([])
labels =  np.array([])
for x, y in test_dataset:
  predictions = np.concatenate([predictions, np.argmax(model.predict(x), axis = -1)])
  labels = np.concatenate([labels, np.argmax(y.numpy(), axis=-1)])

In [ ]:
# 混同行列を表示します
cm = confusion_matrix(labels, predictions)
pd.DataFrame(cm,columns=["pred_" + str(n) for n in class_names], index=["GT_" + str(n) for n in class_names])

In [ ]:
# 行ごとに割合にして表示してみましょう。
pd.DataFrame(cm,columns=["pred_" + str(n) for n in class_names], index=["GT_" + str(n) for n in class_names]).apply(lambda x:x/sum(x),axis=1)

In [ ]:
# 次に各々のクラスについて、precision, recall, f1-scoreを表示してみましょう。
print(classification_report(labels, predictions, target_names=class_names))

### 課題 もっと学習させてみましょう 


1.   base_model (ResNet50) の全ての層を学習可能に
2.   最適化関数はNadam (tf.keras.optimizers.Nadam)
3.   学習率はこれまでの10分の1に
4.   追加のepoch数は20epoch